
## Radio Pulmonaires pour l'analyse du COVID19
### DataScientest : Bootcamp DataScientist
Octobre 2024

#### On commence par importer les packages qui seront utiles
(Il faut les installer préalablement opencv-python  -> cv2
read_excel de pandas a une dépendance vers openpyxl)

In [ ]:
# Import des packages

# Naviguer dans les répertoire

import os

## Data preprocessing
import numpy as np
import pandas as pd 
import string

# Data_viz

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Gestion des images
import cv2


# On indique le répertoire de travail

path = r"C:\Users\Inrae\Documents\Projet_Data_Science\COVID-19_Radiography_Dataset"



Les données ont été téléchargées préalablement. 
Elles sont dispo sous : https://www.kaggle.com/code/ahmedtronic/covid-19-radiology-vgg19-f1-score-95

Il y a trois dossiers (COVID, Lung_Opacity et Normal) contenant chacun plusieurs milliers d'images et les masques associés
Le fichier excel donne des informations sur les metadaonnées : url de la source

Il faudra les importer, les annoter en fonction de leur dossier initial, les préprocesser pour pouvoir les exploiter ensuite en classification

# On explore pour une image


In [ ]:
img_color = cv2.imread(r"./COVID-19_Radiography_Dataset\Lung_Opacity\images\Lung_Opacity-10.png",cv2.IMREAD_COLOR)

print("Le format de l'image est : ", img_color.shape)

plt.imshow(img_color)


In [ ]:

img = cv2.imread(r"./COVID-19_Radiography_Dataset\Lung_Opacity\images\Lung_Opacity-10.png")

print("Le format de l'image est : ", img.shape)


# Split
red = img[:, :, 0]
green = img[:, :, 1]
blue = img[:, :, 2]



colormap = cv2.applyColorMap(img, cv2.COLORMAP_BONE)

fig, ((ax1, ax2,ax3, ax4)) = plt.subplots(ncols=4, nrows=1)
ax1.imshow(img)

ax2.imshow(red, cmap='Reds')

ax3.imshow(green, cmap='Greens')

ax4.imshow(blue, cmap="Blues")


for ax in fig.get_axes():
    ax.label_outer(),
    ax.ticks="",
    ax.axis('off')

#Visuellement il semble y avoir peu de différence entre l'image en nuance de gris ou en couleur, on part dans un premier temps sur les données en gris


In [ ]:
 # On lit le masque correspondant et on l'applique

mask_grey = cv2.imread(r"./COVID-19_Radiography_Dataset\Lung_Opacity\masks\Lung_Opacity-10.png")

print("Le format du masque est : ", mask_grey.shape)

plt.imshow(mask_grey,cmap="gray")


In [ ]:

# On applique le masque 

# On redimensionne le masque au format de l image car l'un est en 256*256 l'autre en 299*299

mask_grey2 = cv2.resize(mask_grey, dsize = img_grey.shape[:2])

# 
masked_image = cv2.bitwise_and(img_grey, mask_grey2) # pour masquer mon_image

plt.imshow(masked_image,cmap="gray")

In [ ]:
# On essaye pour voir s'il ne vaut pas mieux redimensionner l'image 

img_grey2 = cv2.resize(img_grey, dsize = mask_grey.shape[:2])

# 
masked_image2 = cv2.bitwise_and(img_grey2, mask_grey) # pour masquer mon_image

plt.imshow(masked_image2,cmap="gray")

In [ ]:
# On applique une normalisation de gauss

gauss_image = cv2.GaussianBlur(masked_image, ksize = (3,3), sigmaX=0.2)

plt.imshow(gauss_image,cmap="gray")

# Peut être serait il utile de passer en negatif
# gauss_image_neg = 255 - gauss_image
# plt.imshow(gauss_image_neg,cmap="gray")

# A creuser pour s avoir s il serait utile d'utilise erode pour supprimer les electrodes
# ou threshold pour supprimer ce qui est trop blanc ?

Il faut a present stocker l image dans un vecteur

In [ ]:
# On la transforme en format 

a = np.arange(6).reshape((3, 2))

print(a)

a.reshape(6)

# On transforme en vecteur 
gauss_vector = gauss_image.reshape(1,gauss_image.shape[0]*gauss_image.shape[1])

gauss_df  = pd.DataFrame(gauss_vector)

gauss_df["id"] = ("lung_opacity10")
gauss_df["type"] = ("lung_opacity")
#gauss_df["id","type"]= ("lung opacity"

gauss_df.set_index("id")

#gauss_line.append([gauss_vector])

gauss_df


In [ ]:
# On stocle le df de lung opacity

df_lung_opacity = df

# Import de l'ensemble des données

## On importe les images en les preprocessant
### Application du masque
### Application d'un filtre gaussien
### On transforme le tout en dataset


In [ ]:
df  = None

for type in  ["Lung_Opacity","COVID","Normal","Viral_Pneumonia"] :
    print(type)
    list = os.listdir(os.path.join("./COVID-19_Radiography_Dataset",type,"images"))
    #print(list[:10])

   # On peut eventuellement reduire le dataset en prenant max 4000 images par groupe
   #  if len(list)>4000 :
   #     new_list = list[0:2000] + (list[-2000:])
   #  else:
   #     new_list = list
    for filename in list :
        #print(type,filename,os.path.join("./COVID-19_Radiography_Dataset",type,"images",filename))
        img = cv2.imread(os.path.join("./COVID-19_Radiography_Dataset",type,"images",filename),cv2.IMREAD_GRAYSCALE) # import de l'image
        mask = cv2.imread(os.path.join("./COVID-19_Radiography_Dataset",type,"masks",filename),cv2.IMREAD_GRAYSCALE) # import du masque 
        mask2 = cv2.resize(mask, dsize = img.shape[:2])  # pour remettre les dimensions du masque à celle de l'image
        masked_img = cv2.bitwise_and(img, mask2) # pour masquer l'image
        #plt.imshow(masked_img,cmap="gray")
        gauss_img = cv2.GaussianBlur(masked_img, ksize = (3,3), sigmaX=0.2)
        gauss_img2 = cv2.resize(gauss_img, dsize = (100,100))
        gauss_df = pd.DataFrame(gauss_img2.reshape(1,gauss_img2.shape[0]*gauss_img2.shape[1]),index = [filename.split(".")[0]])
        #gauss_df["id"] = filename.split(".")[0]
        gauss_df["type"] = type
        
        if df is None :
           df  = gauss_df
        
        else:
        
           df = pd.concat([df, gauss_df])


    

In [ ]:
df.shape

# listeA = list[0:2000]
# listeB = list[-2000:]
# listeC = listeA + listeB

# #print(listeA)
# #print(listeB)
# tail(listeC)


# On importe les metadonnées 

In [ ]:
# ! pip install openpyxl

all_meta = None

path

for type in  ["Lung_Opacity","COVID","Normal","Viral_Pneumonia"] :
    print(type)
    namefile= type + ".metadata" + ".xlsx"
    file = os.path.join(path,namefile)
    print(file)
    meta = pd.read_excel(file)
    if all_meta is None :
        all_meta  = meta
    else:
        all_meta = pd.concat([all_meta, meta])

# On recree une colonne types et une colonne numéro
# Car normal est ici ecrit en majuscule

all_meta["TYPE"] = all_meta["FILE NAME"].str.split("-", expand=True)[0]
all_meta["num"] = all_meta["FILE NAME"].str.split("-", expand=True)[1]


In [ ]:
# On explore les metadata
display(all_meta.head())
display(all_meta.info())

# On affiche les nombres d occurences
all_meta["FORMAT"].value_counts()
all_meta["SIZE"].value_counts()
all_meta["URL"].value_counts()
all_meta["TYPE"].value_counts()

# On renomme la variable URL
all_meta.replace({
"https://www.kaggle.com/c/rsna-pneumonia-detection-challenge/data" : "rnsa",
"https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia" : "pneumonia-chestxray",                     
"https://bimcv.cipf.es/bimcv-projects/bimcv-covid19/#1590858128006-9e640421-6711"  : "bimcv",
"https://github.com/armiro/COVID-CXNet" : "CXNet",
"https://eurorad.org" : "eurorad",                                                            
"https://github.com/ml-workgroup/covid-19-image-repository/tree/master/png"  : "ml-workgroup",
"https://github.com/ieee8023/covid-chestxray-dataset" : "covid-chestxray",
"https://sirm.org/category/senza-categoria/covid-19/" : "senza",
"NORMAL" : "Normal"
},
inplace = True
)

print(all_meta["URL"].value_counts())

print(all_meta["TYPE"].value_counts())

# On remet un nom coherent

all_meta["FILENAME"] = all_meta["TYPE"] +  "-" +  all_meta["num"]

all_meta =  all_meta.drop("FILE NAME", axis=1)
all_meta = all_meta.set_index("FILENAME")

display(all_meta.head())

# On verifie une eventuelle confusion entre url et type

tableau = pd.crosstab(all_meta["URL"], all_meta["TYPE"])
display(tableau)


## Comptage du nombre d'image par catégorie et par source

In [ ]:
# On fait un comptage du nombre de données par url et par type

sns.countplot(data=all_meta, x="TYPE", hue="URL")
plt.legend(loc='best')

In [ ]:
# On fait un comptage du nombre de données par url et par type
plt.figure(figsize=(10,10))
sns.countplot(data=all_meta, x="TYPE", hue="URL", dodge=False)
plt.legend(loc='best')


fig, ((ax1, ax2)) = plt.subplots(ncols = 2, nrows = 1, figsize=(20,10))
fig.suptitle('Origine des données et catégories')

sns.countplot(data=all_meta,ax=ax1, x="TYPE", hue="URL", dodge=False)
plt.legend(loc='best')
ax1.set_title("Ensemble des données")

dataCOVID = all_meta[all_meta["TYPE"]=="COVID"]
sns.countplot(data=dataCOVID, ax=ax2,  x="TYPE", hue="URL")
ax2.set_title("Focus sur la catégorie COVID")
plt.legend(loc='best')


In [ ]:
# On fusionne les metadata avec les data

all = all_meta.merge(right=df, how= "inner", left_index=True, right_index=True)

# On supprime type qui se retrouve en doublon
all = all.drop("type",axis=1)

display(all.head())

# On exporte le jeu de données complets
all.to_csv("radio_tab.csv") 


## Visualisation des images moyennes

In [ ]:
# # On enleve les variables descriptives

all["TARGET"] = all["URL"] + "_" + all["TYPE"]

X = all.loc[:,0:].drop("TARGET",axis=1)
y = all.TYPE

X.head()

# # # On fait une image moyenne par type 

i=1
for i in range(4) :
    # Sélection des lignes de X_train correspondant au label i
    type = ["Lung_Opacity","COVID","Normal","Viral Pneumonia"][i]
    t = X[y == type]
    # Calcul de l'image moyenne
    moy_img = t.mean(axis=0)
    img =  moy_img.values.reshape(100,100)
    # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
    # à 2 lignes et 2 colonnes. 
    #plt.figure(figsize=(20,20))  
    plt.subplot(2, 2, i+1)
    plt.axis('off')
    plt.imshow(img, cmap="gray", interpolation='None')
    plt.title(type)
    #print(img)
# img



In [ ]:

# # On fait une moyenne par source 

all["TARGET"] = all["TYPE"] + "_" + all["URL"]

url_type = all.TARGET



plt.figure(figsize=(20,20))

print(all.TARGET.nunique())

i=1
j=1
for type in all.TYPE.unique() :
    #plt.subplot(3, 5, i+1)
    N = all[all.TYPE == type].URL.nunique()
    j = 1
    print(N)
    plt.figure(figsize=(5,5*N))
    print(type, N, all[all.TYPE == type].URL.unique())
    for url in all[all.TYPE == type].URL.unique():
        target = type + "_" + url
        t = X[url_type == target]
        # Calcul de l'image moyenne
        moy_img = t.mean(axis=0)
        img =  moy_img.values.reshape(100,100)
        # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
        # à 2 lignes et 2 colonnes. 
        #plt.figure(figsize=(20,20))  
        plt.subplot(1, N, j)
        plt.axis('off')
        plt.imshow(img, cmap="gray", interpolation='None')
        plt.title(target)
        j+=1
        print(target)


plt.show()
# img

In [ ]:
for type in ["Normal"] :
    #plt.subplot(3, 5, i+1)
    N = all[all.TYPE == type].URL.nunique()
    j = 1
    print(N)
    plt.figure(figsize=(5*N,5))
    print(type, N, all[all.TYPE == type].URL.unique())
    for url in all[all.TYPE == type].URL.unique():
        target = type + "_" + url
        t = X[url_type == target]
        # Calcul de l'image moyenne
        moy_img = t.mean(axis=0)
        img =  moy_img.values.reshape(100,100)
        # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
        # à 2 lignes et 2 colonnes. 
        #plt.figure(figsize=(20,20))  
        plt.subplot(N, 1, j)
        plt.axis('off')
        plt.imshow(img, cmap="gray", interpolation='None')
        plt.title(target)
        j+=1
        print(target)

plt.show()



In [ ]:
for type in ["COVID"] :
    #plt.subplot(3, 5, i+1)
    N = all[all.TYPE == type].URL.nunique()
    j = 1
    print(N)
    plt.figure(figsize=(20*N,20))
    print(type, N, all[all.TYPE == type].URL.unique())
    for url in all[all.TYPE == type].URL.unique():
        target = type + "_" + url
        t = X[url_type == target]
        # Calcul de l'image moyenne
        moy_img = t.mean(axis=0)
        img =  moy_img.values.reshape(100,100)
        # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
        # à 2 lignes et 2 colonnes. 
        #plt.figure(figsize=(20,20))  
        plt.subplot(N, 1, j)
        plt.axis('off')
        plt.imshow(img, cmap="gray", interpolation='None')
        plt.title(target)
        j+=1
        print(target)

plt.show()


In [ ]:
for type in ["Lung_Opacity"] :
    #plt.subplot(3, 5, i+1)
    N = all[all.TYPE == type].URL.nunique()
    j = 1
    print(N)
    plt.figure(figsize=(5*N,5))
    print(type, N, all[all.TYPE == type].URL.unique())
    for url in all[all.TYPE == type].URL.unique():
        target = type + "_" + url
        t = X[url_type == target]
        # Calcul de l'image moyenne
        moy_img = t.mean(axis=0)
        img =  moy_img.values.reshape(100,100)
        # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
        # à 2 lignes et 2 colonnes. 
        #plt.figure(figsize=(20,20))  
        plt.subplot(1, N, j)
        plt.axis('off')
        plt.imshow(img, cmap="gray", interpolation='None')
        plt.title(target)
        j+=1
        print(target)

plt.show()

In [ ]:
for type in ["Viral Pneumonia"] :
    #plt.subplot(3, 5, i+1)
    N = all[all.TYPE == type].URL.nunique()
    j = 1
    print(N)
    plt.figure(figsize=(5*N,5))
    print(type, N, all[all.TYPE == type].URL.unique())
    for url in all[all.TYPE == type].URL.unique():
        target = type + "_" + url
        t = X[url_type == target]
        # Calcul de l'image moyenne
        moy_img = t.mean(axis=0)
        img =  moy_img.values.reshape(100,100)
        # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
        # à 2 lignes et 2 colonnes. 
        #plt.figure(figsize=(20,20))  
        plt.subplot(1, N, j)
        plt.axis('off')
        plt.imshow(img, cmap="gray", interpolation='None')
        plt.title(target)
        j+=1
        print(target)

plt.show()

In [ ]:
# # On fait une moyenne par source * type 
N = all.URL.nunique()
list = all.URL.unique()
source = all.URL

print(N, list)
i=1
for i in range(all.URL.nunique()) :
    # Sélection des lignes de X_train correspondant au label i
    url = all.URL.unique()[i]
    t = X[source == url]
    # Calcul de l'image moyenne
    moy_img = t.mean(axis=0)
    img =  moy_img.values.reshape(100,100)
    # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
    # à 2 lignes et 2 colonnes. 
    #plt.figure(figsize=(20,20))  
    plt.subplot(2, 4, i+1)
    plt.axis('off')
    plt.imshow(img, cmap="gray", interpolation='None')
    plt.title(url)
    #print(img)
# img


In [ ]:
# # On fait une moyenne par  type 
N = all.TYPE.nunique()
list = all.TYPE.unique()
source = all.TYPE

print(N, list)
i=1
for i in range(all.TYPE.nunique()) :
    # Sélection des lignes de X_train correspondant au label i
    url = all.TYPE.unique()[i]
    t = X[source == url]
    # Calcul de l'image moyenne
    moy_img = t.mean(axis=0)
    img =  moy_img.values.reshape(100,100)
    # Affichage de l'image dans le i+1-ème emplacement d'une grille de figures
    # à 2 lignes et 2 colonnes. 
    #plt.figure(figsize=(20,20))  
    plt.subplot(2, 2, i+1)
    plt.axis('off')
    plt.imshow(img, cmap="gray", interpolation='None')
    plt.title(url)
    #print(img)
# img
plt.show()

## On supprime les variables identiques pour tout le monde

In [ ]:

#display(data.describe())

# On voit si on peut eliminer les colonnes non variables
data = df.loc[:, (df != df.iloc[0]).any()] 

print("On elimine", df.shape[1] - data.shape[1], "variables identiques pour tous les échantillons")
# ça ne vaut donc pas réellement le coup...




## Dispersion des données par groupe

In [ ]:
# On fait les moyennes et écart types pour chaque ligne 

stat = X.agg(["mean", "std"], axis=1)

In [ ]:
sns.relplot(data=stat, x = "mean", y="std", col = all.TYPE, hue = all.TYPE, hue_order=["COVID","Lung_Opacity","Normal","Viral Pneumonia"], col_wrap=2)
plt.show()

## Nombre de données par catégories, moyennes et écart-type

In [ ]:
# On crée un jeu de données ne contenant plus que la catégorie et les pixels
data = all.drop(["FORMAT", "SIZE", "URL", "num", "TARGET"], axis=1)

# On compte le nombre d'occurrence par groupe

sns.countplot(data=data,x="TYPE",hue="TYPE")
plt.title("Répartition des données par groupe")

# On fait la moyenne des valeurs et écart type par groupe
print("Moyenne des valeurs par catégorie")
display(data.groupby("TYPE").mean().mean(axis=1))
print("Ecart type des moyennes par catégorie")
display(data.groupby("TYPE").mean().std(axis=1))


## Visualiser une différence dans les valeurs des différents pixels selon les catégories
On trace un graph de la moyenne par pixel par groupe

In [ ]:
# On groupe par catégorie et on fait une moyenne par pixel

data_grouped = data.groupby("TYPE").agg("mean")

data_grouped.head()


# On met les catégories comme variables

group2 = data_grouped.unstack().reset_index()

display(group2.head())

In [ ]:
sns.relplot(data=group2, x = "level_0", y= 0, col='TYPE',hue="TYPE", col_wrap=2)
plt.xlabel("pixel")
plt.ylabel("valeur moyenne")

plt.show()

On visualise sous forme de boxplot

In [ ]:
# Autre option on transpose ce qui permet de gérer les données plus facilement
moyenne = data_grouped.transpose()

display(moyenne.head())

In [ ]:
sns.boxplot(moyenne)

display(moyenne.describe())

In [ ]:
sns.boxenplot(moyenne)